# Gpt-image-1.5 プロンプトガイド

## 1. はじめに

`gpt-image-1.5`は、プロダクション品質のビジュアルと高度に制御可能なクリエイティブワークフローのために設計された、私たちの最新の画像生成モデルです。リアリズム、精度、編集可能性において大幅な改善を実現し、プロフェッショナルなデザインタスクと反復的なコンテンツ作成の両方に適しています。前世代と比較してリアリズム、精度、編集可能性において大幅な改善を実現し、高品質レンダリングと低レイテンシーの両方のユースケースをサポートします。

主要な機能には以下が含まれます：

- **高忠実度フォトリアリズム** - 自然な照明、正確な材質、豊かな色彩表現
- **柔軟な品質・レイテンシーのトレードオフ** - 低設定でより高速な生成を可能にしながら、前世代の画像モデルの視覚品質を上回る
- **堅牢な顔と身元の保持** - 編集、キャラクターの一貫性、マルチステップワークフローに対応
- **信頼性の高いテキストレンダリング** - 鮮明な文字、一貫したレイアウト、画像内での強いコントラスト
- **複雑な構造化ビジュアル** - インフォグラフィック、図表、マルチパネル構成を含む
- **精密なスタイル制御とスタイル転送** - 最小限のプロンプトで、ブランドデザインシステムからファインアートスタイルまであらゆるものをサポート
- **強力な実世界の知識と推論** - オブジェクト、環境、シナリオの正確な描写を可能にする

このガイドでは、実際のプロダクション使用例から得られたプロンプトパターン、ベストプラクティス、サンプルプロンプトを紹介します。

## 2. プロンプトの基本原則

* **構造 + 目標:** プロンプトを一貫した順序で記述し（背景/シーン → 被写体 → 重要な詳細 → 制約）、意図された用途（広告、UIモック、インフォグラフィック）を含めて「モード」と完成度のレベルを設定する。複雑なリクエストの場合は、長い段落ではなく、短いラベル付きセグメントや改行を使用する。

* **具体性 + 品質の手がかり:** 素材、形状、質感、視覚的媒体（写真、水彩画、3Dレンダリング）について具体的に記述し、必要な場合のみ的確な「品質レバー」を追加する（例：*フィルムグレイン*、*テクスチャのあるブラシストローク*、*マクロディテール*）。フォトリアリズムの場合、カメラ/構図用語（レンズ、絞り感、照明）は、一般的な「8K/超詳細」よりもリアリズムをより確実に導く。

* **レイテンシ vs 忠実度**: レイテンシに敏感な用途や大量処理の場合は、まずquality="low"に設定して、視覚的要件を満たすかどうかを評価する。多くの場合、大幅に高速な生成で十分な忠実度を提供する。

* **構図:** フレーミングと視点（クローズアップ、ワイド、俯瞰）、パースペクティブ/角度（アイレベル、ローアングル）、照明/ムード（ソフト拡散、ゴールデンアワー、ハイコントラスト）を指定してショットをコントロールする。レイアウトが重要な場合は、配置を明確に指定する（例：「ロゴは右上」、「被写体は中央で左側にネガティブスペース」）。

* **制約（変更するものと保持するもの）:** 除外事項と不変要素を明示的に記述する（例：「透かしなし」、「余分なテキストなし」、「ロゴ/商標なし」、「アイデンティティ/ジオメトリ/レイアウト/ブランド要素を保持」）。編集の場合は、「Xのみ変更」+「他はすべて同じに保つ」を使用し、ドリフトを減らすために各反復で保持リストを繰り返す。

* **画像内のテキスト:** 文字通りのテキストは**引用符**または**すべて大文字**で囲み、タイポグラフィの詳細（フォントスタイル、サイズ、色、配置）を制約として指定する。難しい単語（ブランド名、一般的でないスペル）については、文字精度を向上させるために一文字ずつ綴りを指定する。

* **複数画像の入力:** 各入力を**インデックスと説明**で参照し（「画像1：商品写真… 画像2：スタイル参考…」）、それらがどのように相互作用するかを説明する（「画像2のスタイルを画像1に適用」）。合成する場合は、どの要素がどこに移動するかを明示的に記述する（「画像1の鳥を画像2の象の上に配置」）。

* **過負荷ではなく反復:** クリーンなベースプロンプトから始めて、小さな単一変更のフォローアップで改良する（「照明をより暖かく」、「余分な木を削除」、「元の背景を復元」）。「以前と同じスタイル」や「その被写体」などの参照を使用してコンテキストを活用するが、ドリフトし始めた場合は重要な詳細を再指定する。

## 3. セットアップ

これは一度だけ実行してください。以下の処理を行います：
- APIクライアントを作成
- imagesフォルダ内に`output_images/`を作成
- base64画像を保存するための小さなヘルパーを追加

編集に使用する参照画像は`input_images/`に配置してください（または例のパスを更新してください）。

In [2]:
import os
import base64
from openai import OpenAI

client = OpenAI()

os.makedirs("../../images/input_images", exist_ok=True)
os.makedirs("../../images/output_images", exist_ok=True)

def save_image(result, filename: str) -> None:
    """
    Saves the first returned image to the given filename inside the output_images folder.
    """
    image_base64 = result.data[0].b64_json
    out_path = os.path.join("../../images/output_images", filename)
    with open(out_path, "wb") as f:
        f.write(base64.b64decode(image_base64))

## 4. 使用例 — 生成（テキスト → 画像）

## 4.1 インフォグラフィック
インフォグラフィックを使用して、特定の対象者（学生、経営陣、顧客、または一般大衆）向けに構造化された情報を説明します。例としては、解説図、ポスター、ラベル付きダイアグラム、タイムライン、「ビジュアルwiki」アセットなどがあります。密度の高いレイアウトや画像内テキストが多い場合は、出力生成品質を「高」に設定することを推奨します。

In [3]:
prompt = """
Create a detailed Infographic of the functioning and flow of an automatic coffee machine like a Jura. 
From bean basket, to grinding, to scale, water tank, boiler, etc. 
I'd like to understand technically and visually the flow.
"""

result = client.images.generate(
    model="gpt-image-1.5",
    prompt=prompt
)

save_image(result, "infographic_coffee_machine.png")

![](../../images/infographic_coffee_machine.png)

## 4.2 画像内の翻訳

既存のデザイン（広告、UIスクリーンショット、パッケージ、インフォグラフィック）をレイアウトを一から再構築することなく他の言語にローカライズするために使用されます。重要なのは、テキスト以外のすべてを保持することです。つまり、タイポグラフィのスタイル、配置、間隔、階層を一貫して保ちながら、逐語的かつ正確に翻訳し、余分な単語を追加せず、必要でない限りリフローを行わず、ロゴ、アイコン、画像に意図しない編集を加えないことです。

In [4]:
prompt = """
Translate the text in the infographic to Spanish. Do not change any other aspect of the image.
"""

result = client.images.edit(
    model="gpt-image-1.5",
    image=[
        open("../../images/output_images/infographic_coffee_machine.png", "rb"),
    ],
    prompt=prompt
)


save_image(result, "infographic_coffee_machine_sp.png")

出力画像：

![](../../images/infographic_coffee_machine_sp.png)

## 4.3 「自然」に感じられるフォトリアリスティックな画像

信頼できるフォトリアリズムを得るには、実際の写真がその瞬間に撮影されているかのようにモデルにプロンプトを与えます。写真撮影の用語（レンズ、照明、フレーミング）を使用し、リアルなテクスチャ（毛穴、しわ、生地の摩耗、不完全さ）を明示的に求めます。スタジオでの磨き上げや演出を暗示する言葉は避けてください。ディテールが重要な場合は、`quality="high"`を設定します。

In [6]:
prompt = """
Create a photorealistic candid photograph of an elderly sailor standing on a small fishing boat. 
He has weathered skin with visible wrinkles, pores, and sun texture, and a few faded traditional sailor tattoos on his arms. 
He is calmly adjusting a net while his dog sits nearby on the deck. Shot like a 35mm film photograph, medium close-up at eye level, using a 50mm lens. 
Soft coastal daylight, shallow depth of field, subtle film grain, natural color balance. 
The image should feel honest and unposed, with real skin texture, worn materials, and everyday detail. No glamorization, no heavy retouching. 
"""

result = client.images.generate(
    model="gpt-image-1.5",
    prompt=prompt,
    quality="high"
)

save_image(result, "photorealism.png")

出力画像：

![](../../images/photorealism.png)

## 4.4 世界知識

GPT-image-1.5は組み込まれた推論能力と強力な世界知識を持っています。例えば、1969年8月のニューヨーク州ベセルを舞台とした場面を生成するよう求められた場合、ウッドストックを推測し、そのイベントについて明示的に説明されなくても、正確で文脈に適した画像を生成することができます。

In [7]:
prompt = """
Create a realistic outdoor crowd scene in Bethel, New York on August 16, 1969.
Photorealistic, period-accurate clothing, staging, and environment.
"""

result = client.images.generate(
    model="gpt-image-1.5",
    prompt=prompt
)

save_image(result, "world_knowledge.png")

出力画像：

![](../../images/world_knowledge.png)

## 4.5 ロゴ生成

優れたロゴ生成は、明確なブランド制約とシンプルさから生まれます。ブランドの個性と使用用途を説明し、強い形状、バランスの取れたネガティブスペース、サイズ間でのスケーラビリティを持つクリーンでオリジナルなマークを求めてください。

生成したいバリエーションの数を示すために、パラメータ「n」を指定することができます。

In [8]:
prompt = """
Create an original, non-infringing logo for a company called Field & Flour, a local bakery. 
The logo should feel warm, simple, and timeless. Use clean, vector-like shapes, a strong silhouette, and balanced negative space. 
Favor simplicity over detail so it reads clearly at small and large sizes. Flat design, minimal strokes, no gradients unless essential. 
Plain background. Deliver a single centered logo with generous padding. No watermark.
"""

result = client.images.generate(
    model="gpt-image-1.5",
    prompt=prompt,
    n=4     # Generate 4 versions of the logo
)

# Save all 4 images to separate files
for i, item in enumerate(result.data, start=1):
    image_base64 = item.b64_json
    image_bytes = base64.b64decode(image_base64)
    with open(f"../../images/output_images/logo_generation_{i}.png", "wb") as f:
        f.write(image_bytes)

出力画像：

| オプション 1 | オプション 2 | オプション 3 | オプション 4 |
|:--------:|:--------:|:--------:|:--------:|
| ![](../../images/logo_generation_1.png) | ![](../../images/logo_generation_2.png) | ![](../../images/logo_generation_3.png) | ![](../../images/logo_generation_4.png)|

## 4.6 ストーリーからコミックストリップへ

ストーリーからコミック生成を行う場合、物語を明確な視覚的な場面の連続として定義し、1パネルにつき1つの場面とします。モデルがストーリーを読みやすく、適切なペースのパネルに変換できるよう、説明は具体的でアクション重視にしてください。

In [9]:
prompt = """
Create a short vertical comic-style reel with 4 equal-sized panels.
Panel 1: The owner leaves through the front door. The pet is framed in the window behind them, small against the glass, eyes wide, paws pressed high, the house suddenly quiet.
Panel 2: The door clicks shut. Silence breaks. The pet slowly turns toward the empty house, posture shifting, eyes sharp with possibility.
Panel 3: The house transformed. The pet sprawls across the couch like it owns the place, crumbs nearby, sunlight cutting across the room like a spotlight.
Panel 4: The door opens. The pet is seated perfectly by the entrance, alert and composed, as if nothing happened.
"""

result = client.images.generate(
    model="gpt-image-1.5",
    prompt=prompt
)

save_image(result, "comic_reel.png")

出力画像：

![](../../images/comic-reel.png)

## 4.7 UIモックアップ

UIモックアップは、製品がすでに存在するかのように説明する際に最も効果的に機能します。レイアウト、階層、スペーシング、実際のインターフェース要素に焦点を当て、コンセプトアート的な表現は避けることで、デザインスケッチではなく、実用的で出荷可能なインターフェースのような結果を得ることができます。

In [11]:
prompt = """
Create a realistic mobile app UI mockup for a local farmers market. 
Show today’s market with a simple header, a short list of vendors with small photos and categories, a small “Today’s specials” section, and basic information for location and hours. 
Design it to be practical, and easy to use. White background, subtle natural accent colors, clear typography, and minimal decoration. 
It should look like a real, well-designed, beautiful app for a small local market. 
Place the UI mockup in an iPhone frame.
"""

result = client.images.generate(
    model="gpt-image-1.5",
    prompt=prompt
)

save_image(result, "ui_farmers_market.png")

出力画像：

![](../../images/ui_farmers_market.png)

## 5. ユースケース — 編集（テキスト + 画像 → 画像）

## 5.1 スタイル転送

スタイル転送は、参照画像の*視覚的言語*（パレット、テクスチャ、筆致、フィルムグレインなど）を保持しながら、被写体やシーンを変更したい場合に有用です。最良の結果を得るには、一貫性を保つべき要素（スタイルの手がかり）と変更すべき要素（新しいコンテンツ）を明確に記述し、背景、フレーミング、「余分な要素なし」などの厳格な制約を追加して、意図しない変化を防ぐことが重要です。

In [12]:
prompt = """
Use the same style from the input image and generate a man riding a motorcycle on a white background.
"""

result = client.images.edit(
    model="gpt-image-1.5",
    image=[
        open("../../images/input_images/pixels.png", "rb"),
    ],
    prompt=prompt
)

save_image(result, "motorcycle.png")

入力画像：

![](../../images/pixels.png)

出力画像：

![](../../images/motorcycle.png)

## 5.2 バーチャル衣服試着

バーチャル試着は、アイデンティティの保持が重要なeコマースプレビューに最適です。重要なのは、人物（顔、体型、ポーズ、髪、表情）を明示的に固定し、衣服*のみ*の変更を許可することです。そして、リアルな着用感（ドレープ、しわ、遮蔽）と一貫した照明・影を要求し、衣装が貼り付けられたように見えるのではなく、自然に着用されているように見せることが必要です。

In [13]:
prompt = """
Edit the image to dress the woman using the provided clothing images. Do not change her face, facial features, skin tone, body shape, pose, or identity in any way. Preserve her exact likeness, expression, hairstyle, and proportions. Replace only the clothing, fitting the garments naturally to her existing pose and body geometry with realistic fabric behavior. Match lighting, shadows, and color temperature to the original photo so the outfit integrates photorealistically, without looking pasted on. Do not change the background, camera angle, framing, or image quality, and do not add accessories, text, logos, or watermarks.
"""

result = client.images.edit(
    model="gpt-image-1.5",
    image=[
        open("../../images/input_images/woman_in_museum.png", "rb"),
        open("../../images/input_images/tank_top.png", "rb"),
        open("../../images/input_images/jacket.png", "rb"),
        open("../../images/input_images/tank_top.png", "rb"),
        open("../../images/input_images/boots.png", "rb"),
    ],
    prompt=prompt
)

save_image(result, "outfit.png")

入力画像：

| 全身 | アイテム 1 |
|:------------:|:--------------:|
| ![](../../images/woman_in_museum.png)  | ![](../../images/jacket.png) |
| アイテム 2 | アイテム 3 |
| ![](../../images/tank_top.png) | ![](../../images/boots.png) |

出力画像：

<img src="../../images/outfit.png" width="400"/>

## 5.3 描画 → 画像（レンダリング）

スケッチからレンダリングへのワークフローは、元の意図を保ちながら、ラフな描画をフォトリアリスティックなコンセプトに変換するのに最適です。プロンプトを仕様書のように扱い、レイアウトと遠近法を保持し、もっともらしい素材、照明、環境を指定することで*リアリズムを追加*します。創造的な再解釈を避けるために「新しい要素やテキストを追加しない」を含めてください。

In [14]:
prompt = """
Turn this drawing into a photorealistic image.
Preserve the exact layout, proportions, and perspective.
Choose realistic materials and lighting consistent with the sketch intent.
Do not add new elements or text.
"""

result = client.images.edit(
    model="gpt-image-1.5",
    image=[
        open("../../images/input_images/drawings.png", "rb"),
    ],
    prompt=prompt
)

save_image(result, "realistic_valley.png")

入力画像：

![](../../images/drawings.png)

出力画像：

![](../../images/realistic_valley.png)

## 5.4 製品モックアップ（透明背景 + ラベル整合性）

商品抽出とモックアップ準備は、カタログ、マーケットプレイス、デザインシステムで一般的に使用されています。成功は、エッジの品質（クリーンなシルエット、フリンジ/ハローなし）とラベルの完全性（テキストがシャープで変更されない状態を保つ）に依存します。リスタイリングなしでリアリズムを求める場合は、軽いポリッシュのみを要求し、オプションでアルファを尊重する微妙なコンタクトシャドウを追加することをお勧めします。

In [15]:
prompt = """
Extract the product from the input image.
Output: transparent background (RGBA PNG), crisp silhouette, no halos/fringing.
Preserve product geometry and label legibility exactly.
Optional: subtle, realistic contact shadow in the alpha (no hard cut line).
Do not restyle the product; only remove background and lightly polish.
"""

result = client.images.edit(
    model="gpt-image-1.5",
    image=[
        open("../../images/input_images/shampoo.png", "rb"),
    ],
    prompt=prompt
)

save_image(result, "extract_product.png")

入力画像：

![](../../images/shampoo.png)

出力画像：

![](../../images/extract_product.png)

## 5.5 実際のテキストが画像内に含まれるマーケティングクリエイティブ

実際の画像内テキストを含むマーケティングクリエイティブは、迅速な広告コンセプト作成に最適ですが、タイポグラフィには明示的な制約が必要です。正確なコピーを引用符で囲み、逐語的なレンダリング（余分な文字なし）を要求し、配置とフォントスタイルを説明してください。テキストの忠実度が不完全な場合は、プロンプトを厳密に保ち、反復してください。通常、小さな文言やレイアウトの調整により読みやすさが向上します。

In [16]:
prompt = """
Create a realistic billboard mockup of the shampoo on a highway scene during sunset.
Billboard text (EXACT, verbatim, no extra characters):
"Fresh and clean"
Typography: bold sans-serif, high contrast, centered, clean kerning.
Ensure text appears once and is perfectly legible.
No watermarks, no logos.
"""

result = client.images.edit(
    model="gpt-image-1.5",
    image=[
        open("../../images/input_images/shampoo.png", "rb"),
    ],
    prompt=prompt
)

save_image(result, "billboard.png")

入力画像：

![](../../images/shampoo.png)

出力画像：

![](../../images/billboard.png)

## 5.6 照明と天候の変換

異なる雰囲気、季節、時間帯のバリエーション（例：晴天→曇天、昼間→夕暮れ、晴れ→雪）に対応するために写真を再演出する際に使用されます。シーンの構成は維持したまま、環境条件のみを変更することが重要です。照明の方向や質、影、大気、降水、地面の湿り具合などの環境条件を変更する一方で、アイデンティティ、ジオメトリ、カメラアングル、オブジェクトの配置は保持し、元の写真と同じものとして認識できるようにします。

In [17]:
prompt = """
Make it look like a winter evening with snowfall.
"""

result = client.images.edit(
    model="gpt-image-1.5",
    input_fidelity="high", 
    quality="high",
    image=[
        open("../../images/output_images/billboard.png", "rb"),
    ],
    prompt=prompt
)

save_image(result, "billboard_winter.png")


出力画像：

![](../../images/billboard_winter.png)

## 5.7 オブジェクト除去

人物シーン合成は、顔や身元の保持が重要なストーリーボード、キャンペーン、「もしも」のシナリオに有用です。現実的な写真の見た目（自然な照明、信頼できるディテール、映画的なグレーディングなし）を指定してリアリズムを定着させ、被写体について変更してはならない部分を固定します。利用可能な場合、より高い入力品質は、大きなシーン編集中の類似性維持に役立ちます。

In [18]:
prompt = """
Remove the tree logo from the white t-shirt of the man. Do not change anything else.
"""

prompt = """
Remove the red stripes from the white t-shirt of the man. Do not change anything else.
"""

prompt = """
Change the color of thered hat to light blue as velvet. Do not change anything else.
"""


result = client.images.edit(
    model="gpt-image-1.5",
    input_fidelity="high", 
    quality="high",
    image=[
        open("../../images/output_images/man_with_blue_hat.png", "rb"),
    ],
    prompt=prompt
)

save_image(result, "man_with_no_flower.png")

| 元の入力 | 赤いストライプを除去 | 帽子の色を変更 |
|:------------:|:--------------:|:--------------:|
| ![](../../images/man_with_flower.png)  | ![](../../images/man_with_flower_no_stripes.png) |  ![](../../images/man_with_blue_hat.png) |

## 5.8 人物をシーンに挿入する

人物シーン合成は、顔や身元の保持が重要なストーリーボード、キャンペーン、「もしも」のシナリオに有用です。現実的な写真の外観（自然な照明、信頼できるディテール、映画的なグレーディングなし）を指定してリアリズムを固定し、被写体について変更してはならない部分をロックします。利用可能な場合、より高い入力品質は、大きなシーン編集中の類似性維持に役立ちます。

In [3]:
prompt = """
Generate a highly realistic action scene where this person is running away from a large, realistic brown bear attacking a campsite. The image should look like a real photograph someone could have taken, not an overly enhanced or cinematic movie-poster image.
She is centered in the image but looking away from the camera, wearing outdoorsy camping attire, with dirt on her face and tears in her clothing. She is clearly afraid but focused on escaping, running away from the bear as it destroys the campsite behind her.
The campsite is in Yosemite National Park, with believable natural details. The time of day is dusk, with natural lighting and realistic colors. Everything should feel grounded, authentic, and unstyled, as if captured in a real moment. Avoid cinematic lighting, dramatic color grading, or stylized composition.
"""

result = client.images.edit(
    model="gpt-image-1.5",
    input_fidelity="high", 
    quality="high",
    image=[
        open("../../images/input_images/woman_in_museum.png", "rb"),
    ],
    prompt=prompt
)

save_image(result, "scene.png")

出力画像：

![](../../images/scene.png)

## 5.9 マルチ画像参照と合成

複数の入力から要素を組み合わせて、単一の説得力のある画像を作成するために使用されます。すべてを再生成することなく「このオブジェクト/人物をあのシーンに挿入する」ワークフローに最適です。重要なのは、移植する対象（画像2の犬）、配置場所（画像1の女性のすぐ隣）、変更してはいけない部分（シーン、背景、フレーミング）を明確に指定し、照明、視点、スケール、影を合わせて、合成画像が元の写真で自然に撮影されたように見せることです。

In [4]:
prompt = """
Place the dog from the second image into the setting of image 1, right next to the woman, use the same style of lighting, composition and background. Do not change anything else.
"""

result = client.images.edit(
    model="gpt-image-1.5",
    input_fidelity="high", 
    quality="high",
    image=[
        open("../../images/output_images/test_woman.png", "rb"),
        open("../../images/output_images/test_woman_2.png", "rb"),
    ],
    prompt=prompt
)


save_image(result, "test_woman_with_dog.png")

| 画像入力1 | 画像入力2 | 出力 |
|:------------:|:--------------:|:--------------:|
| ![](../../images/test_woman.png)  | ![](../../images/test_woman_2.png) |  ![](../../images/test_woman_with_dog.png) |

## 6. その他の高価値ユースケース

## 6.1 インテリアデザインの「置き換え」（精密編集）
実際の空間で家具や装飾の変更を視覚化する際に、シーン全体を再レンダリングすることなく使用されます。目標は外科的なリアリズムです：編集が再デザインではなく実際の写真のように見えるよう、カメラアングル、照明、影、周囲のコンテキストを保持しながら単一のオブジェクトを置き換えることです。

In [5]:
prompt = """
In this room photo, replace ONLY white with chairs made of wood.
Preserve camera angle, room lighting, floor shadows, and surrounding objects.
Keep all other aspects of the image unchanged.
Photorealistic contact shadows and fabric texture.
"""

result = client.images.edit(
    model="gpt-image-1.5",
    image=[
        open("../../images/input_images/kitchen.jpeg", "rb"),
    ],
    prompt=prompt
)

save_image(result, "kitchen-chairs.png")

| 入力画像 | 出力画像 |
|------------|--------------|
| ![](../../images/kitchen.jpeg) | ![](../../images/kitchen-chairs.png) |

## 6.2 3Dポップアップホリデーカード（プロダクトスタイルモック）
季節のマーケティングコンセプトや印刷プレビューに最適です。触覚的なリアリズム（紙の層、繊維、折り目、柔らかなスタジオ照明）を重視することで、平面的なイラストではなく、撮影された物理的な製品として認識される結果を生み出します。

In [6]:
scene_description = (
    "a cozy Christmas scene with an old teddy bear sitting inside a keepsake box, "
    "slightly worn fur, soft stitching repairs, placed near a window with falling snow outside. "
    "The scene suggests the child has grown up, but the memories remain."
)

short_copy = "Merry Christmas — some memories never fade."

prompt = f"""
Create a Christmas holiday card illustration.

Scene:
{scene_description}

Mood:
Warm, nostalgic, gentle, emotional.

Style:
Premium holiday card photography, soft cinematic lighting,
realistic textures, shallow depth of field,
tasteful bokeh lights, high print-quality composition.

Constraints:
- Original artwork only
- No trademarks
- No watermarks
- No logos

Include ONLY this card text (verbatim):
"{short_copy}"
"""

result = client.images.generate(
    model="gpt-image-1.5",
    prompt=prompt,
)

save_image(result, "christmas_holiday_card_teddy.png")

出力画像：

![](../../images/christmas_holiday_card_teddy.png)

## 6.3 コレクティブルアクションフィギュア / ぬいぐるみキーホルダー（グッズコンセプト）

初期のグッズアイデア出しとピッチビジュアルに使用されます。デザインをオリジナルで権利侵害のないものに保ちながら、プレミアム商品撮影の要素（素材、パッケージング、印刷の鮮明さ）に焦点を当てています。複数のキャラクターやパッケージのバリエーションを素早くテストするのに適しています。

In [7]:
# ---- Inputs ----
character_description = (
    "a vintage-style toy propeller airplane with rounded wings, "
    "a front-mounted spinning propeller, slightly worn paint edges, "
    "classic childhood proportions, designed as a nostalgic holiday collectible"
)

short_copy = "Christmas Memories Edition"

# ---- Prompt ----
prompt = f"""
Create a collectible action figure of {character_description}, in blister packaging.

Concept:
A nostalgic holiday collectible inspired by the simple toy airplanes
children used to play with during winter holidays.
Evokes warmth, imagination, and childhood wonder.

Style:
Premium toy photography, realistic plastic and painted metal textures,
studio lighting, shallow depth of field,
sharp label printing, high-end retail presentation.

Constraints:
- Original design only
- No trademarks
- No watermarks
- No logos

Include ONLY this packaging text (verbatim):
"{short_copy}"
"""

result = client.images.generate(
    model="gpt-image-1.5",
    prompt=prompt,
)

save_image(result, "christmas_collectible_toy_airplane.png")

出力画像：

![](../../images/christmas_collectible_toy_airplane.png)

## 6.4 キャラクター一貫性を保つ児童書アート（マルチ画像ワークフロー）
キャラクターの変化が許容されないマルチページイラストレーションパイプライン向けに設計されています。再利用可能な「キャラクターアンカー」により、環境や物語の変化を可能にしながら、シーン、ポーズ、ページ全体にわたって視覚的な連続性を確保します。

1️⃣ キャラクターアンカー — 再利用可能なメインキャラクターを確立する

目標：キャラクターの外見、プロポーション、衣装、トーンを固定する。

In [8]:
# ---- Inputs ----
prompt = """
Create a children’s book illustration introducing a main character.

Character:
A young, storybook-style hero inspired by a little forest outlaw,
wearing a simple green hooded tunic, soft brown boots, and a small belt pouch.
The character has a kind expression, gentle eyes, and a brave but warm demeanor.
Carries a small wooden bow used only for helping, never harming.

Theme:
The character protects and rescues small forest animals like squirrels, birds, and rabbits.

Style:
Children’s book illustration, hand-painted watercolor look,
soft outlines, warm earthy colors, whimsical and friendly.
Proportions suitable for picture books (slightly oversized head, expressive face).

Constraints:
- Original character (no copyrighted characters)
- No text
- No watermarks
- Plain forest background to clearly showcase the character
"""

# ---- Image generation ----
result = client.images.generate(
    model="gpt-image-1.5",
    prompt=prompt,
)

save_image(result, "childrens_book_illustration_1.png")

出力画像：

![](../../images/childrens_book_illustration_1.png)

2️⃣ ストーリーの続き — キャラクターを再利用し、物語を進める

目標：同じキャラクター、新しいシーン + アクション。
キャラクターの外見は変更されないようにする必要があります。

In [9]:
# ---- Inputs ----
prompt = """
Continue the children’s book story using the same character.

Scene:
The same young forest hero is gently helping a frightened squirrel
out of a fallen tree after a winter storm.
The character kneels beside the squirrel, offering reassurance.

Character Consistency:
- Same green hooded tunic
- Same facial features, proportions, and color palette
- Same gentle, heroic personality

Style:
Children’s book watercolor illustration,
soft lighting, snowy forest environment,
warm and comforting mood.

Constraints:
- Do not redesign the character
- No text
- No watermarks
"""
# ---- Image generation ----
result = client.images.edit(
    model="gpt-image-1.5",
    image=[
        open("../../images/output_images/childrens_book_illustration_1.png", "rb"),  # ← use image from step 1
    ],
    prompt=prompt,
)

save_image(result, "childrens_book_illustration_2.png")


出力画像：

![](../../images/childrens_book_illustration_2.png)

## 結論

このノートブックでは、gpt-image-1.5を使用して、実際の本番環境で通用する高品質で制御可能な画像生成・編集ワークフローを構築する方法を実演しました。このクックブックでは、リアリズム、レイアウト、テキストの精度、アイデンティティの保持を制御するための主要なツールとして、プロンプト構造、明示的な制約、小さな反復的変更を重視しています。インフォグラフィック、フォトリアリズム、UIモックアップ、ロゴから、翻訳、スタイル転送、バーチャル試着、合成、照明変更まで、生成と編集の両方のパターンをカバーしました。すべての例を通じて、このクックブックでは、変更すべき要素と不変に保つべき要素を明確に分離することの重要性、そしてドリフトを防ぐために各反復でそれらの不変要素を再確認することの重要性を強調しています。また、品質と入力忠実度の設定により、使用ケースに応じてレイテンシと視覚的精度の間で意図的なトレードオフを可能にする方法についても強調しました。これらの例は、本番環境の画像ワークフローにgpt-image-1.5を展開するための実用的で再現可能なプレイブックを形成しています。